# AI-Powered Music Generation
This notebook demonstrates an AI-powered music generation system using Recurrent Neural Networks (RNNs) with LSTM layers. The system is capable of composing original music based on the patterns it learns from the provided dataset of audio files.

## Installation
First, let's install the necessary packages.
```python
!pip install numpy pandas tensorflow music21 pydub librosa
```


In [ ]:
!pip install numpy pandas tensorflow music21 pydub librosa


## Step 1: Import Libraries
Let's import the necessary libraries for this project.

In [ ]:
import numpy as np
import pandas as pd
import os
from music21 import converter, instrument, note, chord, stream
from pydub import AudioSegment
import librosa
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from tensorflow.keras.utils import to_categorical

## Step 2: Convert MP3 to WAV
We need to convert the MP3 file to WAV format for easier processing.

In [ ]:
# Convert MP3 to WAV
def convert_mp3_to_wav(mp3_path, wav_path):
    audio = AudioSegment.from_mp3(mp3_path)
    audio.export(wav_path, format="wav")

# Paths
mp3_path = '/content/Kcee - Ojapiano (Official Video).mp3'
wav_path = '/content/temp_audio.wav'

# Convert
convert_mp3_to_wav(mp3_path, wav_path)

## Step 3: Extract Musical Features
We will extract features from the WAV file using the `librosa` library.

In [ ]:
# Extract musical features
def extract_features(wav_path):
    y, sr = librosa.load(wav_path)
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    features = np.concatenate((chroma_stft, rmse, spec_cent, spec_bw, rolloff, zcr, mfcc))
    return features

# Extract features
features = extract_features(wav_path)

## Step 4: Prepare Sequences
We will prepare sequences from the extracted features to be used as input for the model.

In [ ]:
# Prepare sequences
sequence_length = 100
n_vocab = features.shape[0]

network_input = []
network_output = []

for i in range(len(features[0]) - sequence_length):
    seq_in = features[:, i:i + sequence_length]
    seq_out = features[:, i + sequence_length]
    network_input.append(seq_in)
    network_output.append(seq_out)

n_patterns = len(network_input)

network_input = np.reshape(network_input, (n_patterns, sequence_length, n_vocab))
network_input = network_input / np.max(network_input)
network_output = np.reshape(network_output, (n_patterns, n_vocab))
network_output = network_output / np.max(network_output)

## Step 5: Define the Model
We will define and compile the LSTM model.

In [ ]:
# Define the model
model = Sequential()
model.add(LSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='mean_squared_error', optimizer='adam')

# Model summary
model.summary()

## Step 6: Train the Model
We will train the model on the prepared sequences.

In [ ]:
# Train the model
model.fit(network_input, network_output, epochs=10, batch_size=64)

# Save the model
model.save('music_generation_model.h5')

## Step 7: Generate Music
We will use the trained model to generate new music.

In [ ]:
# Generate new music
def generate_notes(model, network_input, n_vocab, sequence_length):
    start = np.random.randint(0, len(network_input)-1)
    pattern = network_input[start]
    prediction_output = []

    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), n_vocab))
        prediction = model.predict(prediction_input, verbose=0)
        prediction_output.append(prediction[0])
        pattern = np.vstack((pattern[1:], prediction))

    return prediction_output

# Example usage
prediction_output = generate_notes(model, network_input, n_vocab, sequence_length)

## Step 8: Convert Prediction to MIDI
We will convert the generated sequence back to a MIDI file.

In [ ]:
# Convert prediction to MIDI file
def create_midi(prediction_output, output_file):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        new_note = note.Note()
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=output_file)

# Example usage
create_midi(prediction_output, 'generated_music.mid')

The generated music will be saved as `generated_music.mid`.